In [1]:
import pandas as pd
import numpy as np

<img src="./ex1.JPG">

First of all I import csv files extracted from the excel sheet.

In [2]:
df_cc = pd.read_csv("./SONY_Exercise_1 - Customer claim.csv", index_col = 'CustomerNR' )
df_sap = pd.read_csv("./SONY_Exercise_1 - SAP.csv",index_col = 'Customer nr.')

After that, we define a function in order to check discrepancies.

In [3]:
def compensation_test(customer_claim,sap):
    customer_claims_ids = customer_claim.groupby(customer_claim.index)\
    ['QTY to compensate','End stock/Sellin'].sum()
    sap_ids = sap.groupby(sap.index)['Processed quantity'].sum()
    
    not_compensated = [(sap_id,"not_compensated") for sap_id 
                            in sap_ids.index
                               if sap_id not in customer_claims_ids.index ]
    
    overcompensated =  [(customer_claims_id,'overcompensated') 
                            for customer_claims_id
                                in customer_claims_ids.index
                                    if customer_claims_id not in sap_ids.index]
    
    wrong_claims = [(customer_claim_id,'wrong_claim') for customer_claim_id
                    in customer_claims_ids.index
                        if (customer_claims_ids.loc[customer_claim_id]['QTY to compensate'] - sap_ids.loc[customer_claim_id] != 0)]
    
    suspicious_compensations =  not_compensated + overcompensated + wrong_claims
    return  suspicious_compensations

In [4]:
compensation_test(df_cc,df_sap)

[(271500357, 'not_compensated'), (272500650, 'wrong_claim')]

Two discrepancies:
- 271500357 doesn't exist on customer claim sheet.
- 272500650 Claimed one unit more than corresponding on SAP registry.

<img src="./ex2.JPG">

First of all I import csv files extracted from the excel sheet.

In [5]:
df_ac = pd.read_csv("./SONY_Exercise_2 - Amounts to be compensated.csv",index_col = 'Model name' )
df_bq = pd.read_csv("./SONY_Exercise_2 - Billing qtys.csv",index_col = 'Material')

I define a function in order to calculate the costs.

In [6]:
def calculate_total_amount(amount_compensated,billing_quantitys):
    
    billing_quantitys['amount'] = [ amount_compensated.loc[index]['Amount/PC (CZK)']*qty
                                   for (index,qty) 
                                       in zip(billing_quantitys.index,billing_quantitys['Bill qty'])]
    
    grouped_billing_quantitys = billing_quantitys.groupby(billing_quantitys.Name)['amount'].sum()
    
    return grouped_billing_quantitys,grouped_billing_quantitys.sum()

In [7]:
calculate_total_amount(df_ac,df_bq)

(Name
 AT COMPUTERS A.S.               8852
 Aaron shop s.r.o.              18620
 BER-KR SERVICES S.R.O.          4767
 CEWE COLOR a.s.                15820
 ELLEX GK S.R.O.                 5497
 Elvia pro, spol. s r.o.        16348
 MEGAPIXEL S.R.O.               20732
 Milan Skoda-FOTO               19556
 SWS A.S.                        8104
 VIDEO klinik, spol. s r. o.     1424
 Name: amount, dtype: int64, 119720)

Total costs is 119720

<img src="./ex3.JPG">

First of all I import csv file extracted from the excel sheet.

In [8]:
df_fy = pd.read_csv("./SONY_Exercise_3 - FY13 Sept-March DI.csv",index_col = 'Bill.Doc.')

I parse 'bill date' colum to datetime type.

In [9]:
df_fy['date'] = pd.to_datetime(df_fy['Bill. Date'])

now we calculate the amount of bills on 2013-12-20

In [10]:
ondate = df_fy.loc[df_fy['date'] == '2013-12-20']

In [11]:
ondate

,Item,Material,MPG,Billed qty,CnTy,Amount,CondRecNo.,Agreement,Cond.value,SOff.,...,Product hierarchy,MG3*,MG4*,Created,OrdRs,Sales Doc.,Doc. Date,Purchase order no.,Pricing dt,date
Bill.Doc.,,,,,,,,,,,,,,,,,,,,,
6876100217,260,SLTA58K.CEC,NaN,2,ZNET,130,NaN,NaN,260,PL00,...,SCPCDIMAMCDSLRDSLR,100.0,NaN,HUHATALA,C03,67873655,17.12.2013,sell out support-MSH,10.04.2013,2013-12-20
6876100218,120,DSCHX300B.CE3,NaN,2,ZNET,70,NaN,NaN,140,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUHATALA,C03,67876966,17.12.2013,sell out support-MSH,13.05.2013,2013-12-20
6876100219,60,DSCHX50B.CE3,NaN,1,ZNET,60,NaN,NaN,60,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUHATALA,C03,67876970,17.12.2013,sell out support-MSH,15.05.2013,2013-12-20
6876100220,70,DSCHX50S.CE3,NaN,2,ZNET,60,NaN,NaN,120,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUHATALA,C03,67876971,17.12.2013,sell out support-MSH,18.09.2013,2013-12-20
6876100221,150,SLTA58K.CEC,NaN,1,ZNET,130,NaN,NaN,130,PL00,...,SCPCDIMAMCDSLRDSLR,100.0,NaN,HUHATALA,C03,67873688,17.12.2013,sell out support-MSH,10.04.2013,2013-12-20
6876100222,110,DSCRX100.CEE8,NaN,1,ZNET,163,NaN,NaN,163,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUHATALA,C03,67876281,17.12.2013,sell out support-MSH,23.11.2012,2013-12-20
6876100223,50,DSCRX100.CEE8,NaN,1,ZNET,163,NaN,NaN,163,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUHATALA,C03,67876284,17.12.2013,sell out support-MSH,03.12.2012,2013-12-20
6876100224,40,DSCWX200B.CE3,NaN,2,ZNET,50,NaN,NaN,100,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUHATALA,C03,67876563,17.12.2013,sell out support-MSH,12.03.2013,2013-12-20
6876100225,60,DSCWX200B.CE3,NaN,2,ZNET,50,NaN,NaN,100,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUHATALA,C03,67876566,17.12.2013,sell out support-MSH,10.04.2013,2013-12-20


481 invoices were created on 20th of December of 2013

In [12]:
krakow_invoices = df_fy.loc[df_fy['Name 1'] == 'Neonet S.A. Krakow']

In [13]:
krakow_invoices

,Item,Material,MPG,Billed qty,CnTy,Amount,CondRecNo.,Agreement,Cond.value,SOff.,...,Product hierarchy,MG3*,MG4*,Created,OrdRs,Sales Doc.,Doc. Date,Purchase order no.,Pricing dt,date
Bill.Doc.,,,,,,,,,,,,,,,,,,,,,
6876107368,60,ILCE3000KB.CE,NaN,59,ZNET,210,NaN,NaN,"12.390,00",PL00,...,SCPCDIMAMCNEXKNEXK,100.0,NaN,HUPERJEK,C03,67999956,11.02.2014,Sales Out allowance,18.11.2013,2014-02-14
6876112253,10,DSCW710B.CE3,NaN,5,ZNET,10,NaN,NaN,50,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUPERJEK,C03,68063693,13.03.2014,FY13 SPOL - NEONET -,29.01.2013,2014-03-14
6876112253,20,DSCW710S.CE3,NaN,5,ZNET,10,NaN,NaN,50,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUPERJEK,C03,68063693,13.03.2014,FY13 SPOL - NEONET -,29.01.2013,2014-03-14
6876112254,131,DSCW710B.CE3,NaN,70,ZNET,10,NaN,NaN,700,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUPERJEK,C03,68063694,13.03.2014,FY13 SPOL - NEONET -,14.02.2013,2014-03-14
6876112255,20,DSCW710B.CE3,NaN,20,ZNET,10,NaN,NaN,200,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUPERJEK,C03,68063695,13.03.2014,FY13 SPOL - NEONET -,13.03.2013,2014-03-14
6876112256,20,DSCW710B.CE3,NaN,10,ZNET,10,NaN,NaN,100,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUPERJEK,C03,68063696,13.03.2014,FY13 SPOL - NEONET -,27.03.2013,2014-03-14
6876112257,20,DSCW710B.CE3,NaN,10,ZNET,10,NaN,NaN,100,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUPERJEK,C03,68063697,13.03.2014,FY13 SPOL - NEONET -,17.04.2013,2014-03-14
6876112258,30,DSCW710S.CE3,NaN,10,ZNET,10,NaN,NaN,100,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUPERJEK,C03,68063698,13.03.2014,FY13 SPOL - NEONET -,12.03.2013,2014-03-14
6876112258,50,DSCW730G.CE3,NaN,1,ZNET,12,NaN,NaN,12,PL00,...,SCPCDIMDSCCYBRCYBS,100.0,NaN,HUPERJEK,C03,68063698,13.03.2014,FY13 SPOL - NEONET -,12.03.2013,2014-03-14


17 invoices were created for Neonet S.A. Krakow

Now we select Billed quantities greather than 60

In [14]:
filtered = df_fy.loc[df_fy['Billed qty'] > 60][['Billed qty','  Cond.value']]

Cond.value column have an invalid format for numbers, so we must transform them.

In [15]:
filtered['  Cond.value'] = [value.replace('.','')for value in filtered['  Cond.value']]
filtered['  Cond.value'] = [value.replace(',','.')for value in filtered['  Cond.value']]
filtered['  Cond.value'] = [float(value) for value in filtered['  Cond.value']]

In [16]:
filtered


,Billed qty,Cond.value
Bill.Doc.,,
6876083708,70,3500.00
6876083808,92,9752.00
6876086991,123,5043.00
6876086992,77,9240.00
6876086998,94,11280.00
6876087834,87,10440.00
6876107370,70,14700.00
6876107730,71,16330.00
6876107739,67,15410.00


In [17]:
filtered.sum()

Billed qty        5385.0
  Cond.value    406696.8
dtype: float64

- Total amount of bills is  5385.0
- Total amount of conditional value is 406696.8